# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing import image_dataset_from_directory

In [2]:
tf.__version__

'2.16.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
training_data = image_dataset_from_directory('dataset/training_set',
                                             image_size=(64, 64),
                                             batch_size=32,
                                             label_mode='binary')

train_datagen = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(0.2, 0.2)
])

train_set = training_data.map(
    lambda x, y: (train_datagen(x, training=True), y)
)
#train_set = train_set.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


Found 8000 files belonging to 2 classes.



### Preprocessing the Test set

In [4]:
test_data = image_dataset_from_directory('dataset/test_set',
                                             image_size=(64, 64),  # Size to which all images will be resized
                                             batch_size=32,
                                             label_mode='binary')

test_datagen = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])

test_set = test_data.map(
    lambda x,y: (test_datagen(x, training= False), y)
)

#test_set = test_set.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Found 2000 files belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size = 3, activation='relu', input_shape = [64,64,3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size = 3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units = 128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

Epoch 1/25


250/250 [==============================] - 35s 79ms/step - loss: 0.6879 - accuracy: 0.5493 - val_loss: 0.6676 - val_accuracy: 0.6045
Epoch 2/25
250/250 [==============================] - 19s 74ms/step - loss: 0.6463 - accuracy: 0.6181 - val_loss: 0.5944 - val_accuracy: 0.6850
Epoch 3/25
250/250 [==============================] - 19s 74ms/step - loss: 0.6131 - accuracy: 0.6600 - val_loss: 0.5972 - val_accuracy: 0.6845
Epoch 4/25
250/250 [==============================] - 19s 75ms/step - loss: 0.5998 - accuracy: 0.6745 - val_loss: 0.6020 - val_accuracy: 0.6695
Epoch 5/25
250/250 [==============================] - 19s 74ms/step - loss: 0.5857 - accuracy: 0.6851 - val_loss: 0.5360 - val_accuracy: 0.7270
Epoch 6/25
250/250 [==============================] - 19s 76ms/step - loss: 0.5698 - accuracy: 0.7003 - val_loss: 0.5370 - val_accuracy: 0.7280
Epoch 7/25
250/250 [==============================] - 19s 75ms/step - loss: 0.5569 - accuracy: 0.7151 - val_loss: 0.5109 - val_accurac

## Part 4 - Making a single prediction

### Printing the clases

In [14]:
classes_indices = training_data.class_names
print(classes_indices)

['cats', 'dogs']


In [15]:
import numpy as np
from keras.preprocessing import image

In [18]:
for i in range(1,5):
    test_image = image.load_img(f"dataset/single_prediction/cat_or_dog_{i}.jpg", target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    print(result)
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'
    print(prediction)

1/1 [==============================] - 0s 43ms/step
[[1.]]
dog
1/1 [==============================] - 0s 39ms/step
[[1.]]
dog
1/1 [==============================] - 0s 39ms/step
[[0.]]
cat
1/1 [==============================] - 0s 36ms/step
[[1.]]
dog


In [17]:
cnn.save('cat_or_dog_model.h5')

C:\Users\sahil\AppData\Roaming\Python\Python311\site-packages\tf_keras\src\engine\training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Save the model in the TensorFlow SavedModel format
cnn.save('my_model')

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets
